In [11]:
#Packages to help with analysis
import matplotlib.pyplot as plt
import nltk, re, tweepy 
import numpy as np
import pandas as pd
import pyreadstat
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from collections import Counter
import sklearn
from textblob import TextBlob
from sklearn.manifold import TSNE
from textblob import Word
from spellchecker import SpellChecker
%matplotlib inline



# Let's Take a Look at the Data

In [12]:
pd.set_option('display.max_columns', None)
df, meta = pyreadstat.read_sav('Emotional Well-Being - CASEL COVID-19 Webinar_Anon.sav')
df.head()

,StartDate,EndDate,Progress,Duration__in_seconds_,Finished,RecordedDate,ResponseId,OpenEmotions_1,OpenEmotions_3,OpenEmotions_4,ClosedEmotions_1,ClosedEmotions_2,ClosedEmotions_3,ClosedEmotions_4,ClosedEmotions_5,ClosedEmotions_6,StressCauses_1,StressCauses_12,StressCauses_13,JoyCauses_1,JoyCauses_12,JoyCauses_13,Strats_EffectiveSelf_1,Strats_EffectiveSelf_2,Strats_EffectiveSelf_3,Strats_EffectiveOthe_1,Strats_EffectiveOthe_2,Strats_EffectiveOthe_3,ER_1,ER_2,WellBeingSupport_16,WellBeingSupport_17,WellBeingSupport_18,CASELWebinars_16,CASELWebinars_17,CASELWebinars_18,Age,Race_1,Race_2,Race_3,Race_4,Race_5,Race_6,Race_7,Race_8,Race_8_TEXT,Gender,Gender_3_TEXT,Educator,Role_11,Role_12,Role_1,Role_2,Role_3,Role_4,Role_5,Role_6,Role_7,Role_8,Role_10,Role_10_TEXT,Grade_1,Grade_2,Grade_3,Grade_4,StudentSES_1
0,2020-03-24 16:47:03,2020-03-24 16:52:06,100.0,302.0,1.0,2020-03-24 16:52:06,R_Q64hfjjdCiGnM09,Anxious,Sad,Fearful,6.0,6.0,3.0,5.0,5.0,3.0,uncertainty,News,updates,kids,work,family,stop soical media,running,journaling,,,,NaN,NaN,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
1,2020-03-24 16:48:25,2020-03-24 16:52:45,100.0,259.0,1.0,2020-03-24 16:52:45,R_Anw03Hj4s7jxnfH,anxiety,fear,hope,5.0,6.0,3.0,4.0,3.0,4.0,worry about others,worry about self,loss of normal routine,connecting with others,,,yoga,connecting with others,,listen to them,provide facts,provide resources,3.0,4.0,Less cognitive demands,Show grace and understanding,Listen to what people need,,,,39,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
2,2020-03-24 16:51:16,2020-03-24 16:52:49,100.0,92.0,1.0,2020-03-24 16:52:49,R_32UYohfSNVrkECi,,,,NaN,NaN,NaN,NaN,NaN,NaN,,,,,,,,,,,,,NaN,NaN,,,,,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN
3,2020-03-24 16:47:38,2020-03-24 16:53:13,100.0,334.0,1.0,2020-03-24 16:53:13,R_24PeWDZ57ldPxMy,Fear,Sadness,Anger,5.0,5.0,4.0,5.0,4.0,3.0,Uncertainty,Change,Finances,Family,Connectedness,Quiet,Breathing,Meditation,Exercise,Calm leadership,Meditation,Optimism,5.0,6.0,Reassurance,Non-education related activities that increase...,"Ways of increasing normalcy, routine, and conn...",,,,36,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,1.0,5.0
4,2020-03-24 16:46:45,2020-03-24 16:53:26,100.0,400.0,1.0,2020-03-24 16:53:26,R_wMCGldjiVHje0o1,Panic,Terror,Depression,3.0,5.0,4.0,4.0,4.0,5.0,Fear for my health,Fear for my family’s health,Social isolation,My son’s smile,My son’s laughter,Old VMs from a friend,Yoga,EFT,Nature,Outside time,Validating feelings,Hugging,5.0,5.0,Acceptance of all feelings,Tools to use in the moment,Space,Access to physical touch,Self reg strategies,EFT,,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,,2.0,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Private SLP,1.0,1.0,NaN,1.0,8.0


# Combine all stress repsonse in one Series all_strats

In [13]:
most_feelings_current_climate = []
##
##Append variables
aux = np.append(most_feelings_current_climate, df['StressCauses_1'])
aux1 = np.append(aux, df['StressCauses_12'])
aux2strat = np.append(aux1, df['StressCauses_13'])

##Create series and lovercase all characters
##/
all_strats = pd.Series(pd.Series([x.lower() for x in aux2strat]))




ordered_counts_strat = all_strats.value_counts()

len(ordered_counts_strat)



9187

In [14]:
def ordered_counts(list_):
    ##takes a list of responses
    ##
    ##Returns the 50 most common responses
    aux = pd.Series(list_)
    ordered_counts_strat = aux.value_counts()
    return ordered_counts_strat[0:50]
  

In [15]:
def response_list(open_questions_haystack, needle):
    ##Takes in the open ended data and finds all instances where a word (Neele) is used.
    ##
    ##Returns a list of those items
    response = []
    for i in open_questions_haystack:
        if i.find(needle) >=0:
            response = np.append(response, i)
            

    return(response)




In [16]:
def category_addition_correct(open_questions_haystack, category):
    total_words = []
    for i in category:        
        response = response_list(open_questions_haystack, i)
        sub_words = []
        for word in response:
            if word in total_words:
                if word in sub_words:
                    total_words =list(np.append(total_words, word))
                
            else:
                sub_words =list(np.append(sub_words, word))
                total_words =list(np.append(total_words, word))

    return total_words

# Categories to get data from.

In [39]:
#Create categories
#

work = ['work','job', 'effort', 'employment' ,'industry', 'obligation', 'profession', 'trade',\
        'opus' ,'occupation' ,'business','composition', 'task', 'vocation', 'endeavor', 'calling', \
        'craft', 'admini', 'students', 'career', 'school', 'colleagues', 'staff', 'employ', 'teach']


uncertainty =['uncertain', 'suspicion', 'incertitude', 'dubiety' , 'doubt' , 'skepticism' ,'mistrust', 'distrust',\
              'indecision', 'dilemma', 'misgiving' ,'conjecture', 'wonder', 'quandary' ,'distrustfulness', \
              'ambivalence', 'vagueness' ,'ambiguity', 'doubtfulness', 'query','puzzle', 'questionableness',\
              'reservation', 'inconclusive', 'unknown', 'change']


family = ["son", 'child', 'fami', 'husb', 'wife', 'daughter', 'kid', 'partner', 'spouse', 'parent', 'mom', 
          'babies', 'loved one', 'niece', 'nephew', 'grandm', 'grandf', 'mother', 'father',
          'newborn', 'sibling', 'girl', 'boy', 'sister', 'brother', 'cousin', 'aunt', 'pregnant']

money = ['bankroll', 'stake', 'fund', 'subsidize', 'underwrite' ,'endow', 'financ', 'money', \
         'capitalize', 'invest', 'subsidy', 'capital' , 'economic', 'banking', 'business', 'dollar', \
         'economy', 'bills', 'income']


fear =['worry', 'scare', 'fright', 'dread' ,'panic', 'terror', 'horror', 'agitation' ,'alarm','phobia','trouble',\
       'fret','cold feet' ,'presentiment','concern','trembling','foreboding','uneas','trepidation','dismay', 'sweat', \
       'misgiving', 'aversion', 'anxi' ,'funk' ,'bother', 'disquietude' ,'doubt', 'suspicion', 'timidity' ,'stress',
       'consternation', 'afraid', 'fear']

sick = ['sick','indisposed','down','ill','unwell','ailing','unhealthy','run-down','poorly','suffering',\
        'in a bad way','infected','bedridden','diseased' ,'impaired','in poor health' ,'laid up',\
        'sickened', 'hospitalized' ,'frail' ,'feeble','feverish','hospitalised', 'sickly', 'covid','virus',\
       'immune system', 'corona','health', 'infect', 'death']
##
##
media = ['magazines', 'communications' 'industry','media','cable','announcing',\
         'newspapers' ,'radio' ,'television' ,'news']

loneliness = ['lonely','solitariness','aloneness','seclusion','solitude','secludedness','loneliness',\
              'separation','detachment', 'quarantine','remoteness', 'alone']

resources = ['food', 'supplies', 'shortage', 'basic need', 'toilet paper', 'tp', 'neccesties','essentials',\
             'grocery', 'resources']




## The number of responses in each category

In [52]:


y = category_addition_correct(list(all_strats), family)
len(y)

3510

In [41]:
def response_to_excel(all_strats, category):
    #takes in a series of all strategies and a list of words in a category
    #
    #
    aux = pd.Series(list(category_addition_correct(list(all_strats), category)))

    df1 = aux.to_frame()
    df2 = aux.value_counts().to_frame()
    writer = pd.ExcelWriter('{}.xlsx'.format(category), engine='xlsxwriter')
    # Write each dataframe to a different worksheet.
    df1.to_excel(writer, sheet_name='{}1'.format(category))
    df2.to_excel(writer, sheet_name='{}2'.format(category))
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [42]:
def uncategorized(all_strats, category):
    #Takes in a series of all strats and a category list
    #
    #returns all things not cclassified in those categories
    category_words = category_addition_correct(list(all_strats), category)
    unique_all_strats = np.unique(all_strats)
    unique_y = np.unique(category_words)

    uncategorized = [x for x in unique_all_strats if x not in unique_y]
    
    return uncategorized


In [43]:
category = work + uncertainty + media + family + money +fear +sick +loneliness +resources

uncategorized_data = uncategorized(all_strats, category)



In [44]:
len(uncategorized_data)/len(all_strats)


0.174578570044565

In [45]:
list(uncategorized_data)

['',
 ' delivery of sel supports',
 ' depressed',
 ' do little exercise',
 " don't leave home",
 ' feelings of hopelessness',
 ' manage processes and people',
 ' not enough confidence if i do everything right',
 ' unsure about our countries rules',
 ' well being of our world',
 '"resource guarding"',
 '...',
 '45',
 '62 years old',
 '?',
 'a',
 'a disturbed at the non serious attitude of the general public, who are not heeding towards the advice of social distancing',
 'a full house with cabin fever',
 'a houseful of adults and young adults',
 'a lot of time spent at home',
 'a lot on my plate',
 'a trip planned',
 'a want to do something but not knowing what',
 'a wedding planned in 4 months',
 'abbott',
 'ability to care for others',
 'ability to focus',
 'ability to travel',
 'abnormal conditions in world',
 'abruptly altering daily life and routines',
 'abundance of conflicting information',
 'access to necessities',
 'access to needed items',
 'access to things',
 'accessibility t

In [54]:
aux = pd.Series(list(uncategorized_data))

df1 = aux.to_frame()
df2 = aux.value_counts().to_frame()
writer = pd.ExcelWriter('{}.xlsx'.format('uncategorized'), engine='xlsxwriter')
    # Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='{}1'.format('uncategorized'))
df2.to_excel(writer, sheet_name='{}2'.format('uncategorized'))
    # Close the Pandas Excel writer and output the Excel file.
writer.save()